<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

註冊Kaggle

sign in with google: ReZeroBen


資料預處理

(Sibsp: 手足 siblings 、配偶 spouse)\
(Parch: 父母Parents、兒女child) \
(Fare: 船費)\
(Embarked: 登船口岸 
C=法國Cherbourg, Q=愛爾蘭Queenstown, S=英國Southampton)\
[ https://chtseng.wordpress.com/2017/12/24/kaggle-titanic%E5%80%96%E5%AD%98%E9%A0%90%E6%B8%AC-1/ ]

欄位先分成兩種
  1. 類別型：艙等 Pclass、Name(mid: Mr. 、Mrs. ..)、Sex(直接換成2值型)、艙號 Cabin(首字母$1^{st}$: 區域)、Embarked登船港口

  2. 數值型: 年紀age、SibSp、Parch、ticket(share同張票)、Fare


1. 填補缺失值：=> 最可能的數值 \
從可能的欄位推敲、\
類別：該欄位最常出現的值

數值：=> ex:薪資->中位數
  - 眾數
  - 平均數(單位:數值、錢、收入...把大家削成一樣的!)
  - $\ast$中位數$\ast$(單位：人數、數量、計數...)

一起填 pandas.concat
[ https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html ]

axis 雙向型的api

axis=0 縱向連接 \
axis=1 橫向型連接(擴增欄位)

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/kingsgameBen/MLbasic_0223/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/kingsgameBen/MLbasic_0223/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")
train_df

In [ ]:
datas = pd.concat([train_df, test_df], axis=0, ignore_index=True)
datas

(dataframe.sum())[ https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html ]

把有缺的值做一個篩選



In [ ]:
# .count()數欄位 .sum()可以縱向橫向相加
s = datas.isna().sum()
# Series[True/False]
# s[s > 0].sort_values()
s[s > 0].sort_values(ascending=False)

In [ ]:
datas = datas.drop(["PassengerId", "Survived"], axis=1)
# 數值類型：中位數
# median() 取datas各個非文字欄位的中位數
# median()排除Pclass欄位, 是因為該欄位的值1, 2, 3是代表等級, 而不是數量
# fillna()會把datas各個欄位裡, 有取得median的欄位(age、Fare)空值都填上
med = datas.median().drop("Pclass")
print(med)
datas = datas.fillna(med)

s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

DataFrame.idxmax() \
[ https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.idxmax.html ] \
對該欄位值分類計數後, 取計數值最大的 => 最常出現的眾數


In [ ]:
# 類別類型： 找最常出現,不可以取中位數
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

In [ ]:
# 處理 Cabin 
def cabin(s):
    if pd.isna(s):
        return None
    else:
        return s[0]
datas["Cabin"] = datas["Cabin"].apply(cabin)


In [ ]:
# 處理 Name
def name(s):
    mid = s.split(",")[-1].split(".")[0].strip()
    return mid
# 去除name裡稀少的樣本
count = datas["Name"].apply(name).value_counts()
reserved = count[:4].index
# print(count, reserved)
def name_reserved(s):
    mid = s.split(",")[-1].split(".")[0].strip()
    if mid in reserved:
        return mid
    else:
        return None
datas["Name"] = datas["Name"].apply(name_reserved)

datas

In [ ]:
# 處理 Ticket
dic = datas["Ticket"].value_counts()
def ticket(t):
    return dic[t]
datas["Ticket"] = datas["Ticket"].apply(ticket)
datas["Ticket"]

get_dummies[ https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html ]
產生虛擬欄位 \
將該欄位row裡的值分類轉換成虛擬欄位,就可以做計數 \

ex: \
Sex: male、female $\Downarrow$ \
Sex_male: 0, 1, 1, 0, 0, 0 ..... \
Sex_female: 1, 0, 0, 1, 1, 1...


### $\color {red}{One-Hot encoding 獨熱編碼}$
當特徵不是連續值,而是分類值時,為了比大小,要把值轉換成 \
數個是非題: 只有0, 1

~{Pclass欄位,在這裏用偷懶的方式,不做one-hot encoding}~



In [ ]:
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])
datas

In [ ]:
# 是否新增欄位, 是靠觀察欄位之間的關係, 不斷嘗試是否能提高預測準確率,得到的結果
# 新增欄位：將手足配偶 旁系欄位 ＋ 雙親兒女 直系欄位 結合成 家族欄位
# (親屬或家人一起可以互相幫助)
datas["Family"] = datas["SibSp"]+datas["Parch"]
datas

隨機森林

Ensemble集成式的演算法

(RandomForestClassifier)[ https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html ]

新版：要調整的參數
1. 森林大小-預設值＝10 n_estimators=10  (api沒更新到) \
2. max_depth:最大深度

(隨機森林regressor:取森林投票平均值) \
每棵樹都放棄一點資料,如每棵決策樹都只取90%而不是吃全部, \
達到每棵樹都略有不同,最終投票的結果才會有意義

調整參數的方法：$\color{lime}{交叉驗證\ Cross \ \ Validation}$ \
(視資料決定要用哪種演算法：決策樹、隨機森林...)

pandas 的 iloc : index location 代表該筆資料在第幾個 Row \
(從0起算)



In [ ]:
y = train_df["Survived"]
# .iloc: [1st row, 2nd row, ....]
x = datas.iloc[:len(train_df)]
predict = datas.iloc[len(train_df):]
# 將處理後的datas, 切割成兩份, x = x_train, predict = x_test
predict

model_selection 調整參數的估算器
[ https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection ]


寫一組loop, 每一組參數都試一遍
Grid Search \
[ https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html ] \

estimator 估算器=> ex:隨機森林分類器 \
param_grid: 字典或字典清單, 給一個區間讓GridSearchCV迭代找出最佳的參數 \
scoring: 評分標準, 這裡使用negative log loss來找到最接近0的參數, 也就是平均最短距離, negative就是評分會以負數表示



In [ ]:
# ensemble 一起、整體 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier()
params = {
    "n_estimators":[11, 12],
    "max_depth":[5, 6]
}
# params = {
#     "n_estimators":range(11, 151, 2),
#     "max_depth":range(5, 11)
# }
grid = GridSearchCV(clf, params, cv=10, n_jobs=-1, scoring='neg_log_loss')
grid.fit(x, y)
print(grid.best_score_)
print(grid.best_params_)

(cross_val_score)[ https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html ]

cv=cross validation 切幾份
交叉驗證的機制 [ https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation ] \


n_jobs 分工同時執行,電腦4核心就同時執行4份工\
預設None 單核心, 設-1則視電腦核心數決定同時跑多少工

scoring 評分機制 \
(model evaluation documentation)
[ https://scikit-learn.org/stable/modules/model_evaluation.html ]



In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=50, max_depth=6)
scores = cross_val_score(clf, x, y, cv=10, n_jobs=-1)
print(scores)
print(np.average(scores))
clf = RandomForestClassifier(n_estimators=53, max_depth=8)
scores = cross_val_score(clf, x, y, cv=10, n_jobs=-1, scoring='neg_log_loss')
print(scores)
print(np.average(scores))

In [ ]:
clf = RandomForestClassifier(n_estimators=53, max_depth=7)
clf.fit(x, y)
pre = clf.predict(predict)
ans = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
ans.to_csv("rf.csv", encoding="utf-8", index=False)

MinMaxScaler: \
[ https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler ] \
preprocessing預處理的class

將特定欄位的值做縮放固定在某個範圍,通常會收縮範圍 0 ~ 1, \
此題對所有欄位縮放到0 ~ 1, \
是為了使用KNN演算法(以 計算距離 為主的演算法), 因為此法沒有隨機性, \
故須先做標準化,讓features的值收縮後,大家都差不多(單位一致), \
此案例 Titanic中的age年紀欄位,若不收縮,會導致主宰其他欄位資訊,使結果失真


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datas_scale = scaler.fit_transform(datas)
datas_scale = pd.DataFrame(datas_scale, columns=datas.columns)
datas_scale

In [ ]:
import pandas as pd
pd.DataFrame(clf.feature_importances_,
             columns=["Importance"],
             index=datas.columns).sort_values(by="Importance", ascending=False)

In [ ]:
dt = clf.estimators_[1]
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(14, 14))
plot_tree(dt, 
     feature_names=datas.columns, 
     class_names=["Dead", "Alive"], 
     max_depth=3,
     filled=True)


### female、male存活『人數』
用Seaborn

知道哪些特徵很重要,才知道畫哪些圖
    1. 散佈圖(連續數值...scatter)
    2. 數量圖
    3. 區間圖

matplotlib參數設定 \
tick：xticks、yticks x軸及y軸上的資料間隔 \
rotation: 間隔的label字太長,要旋轉
[ https://matplotlib.org/stable/gallery/ticks_and_spines/ticklabels_rotation.html ] \
legend: 圖表的簡要label說明 \

pd.cut():將資料的區間切成幾等份
[https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html ]\

seaborn.histplot(): 用不到,因為此方法的hue只有定義顏色,無法設定欄位
[https://seaborn.pydata.org/generated/seaborn.histplot.html#seaborn.histplot ]

sort_values(): by = 欄位 or row, axis: 1, 0
[https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html ]




In [ ]:
import seaborn as sns
sns.countplot(y=train_df["Sex"], hue=train_df["Survived"])

In [ ]:
# sns.histplot(train_df["Fare"])
# print(train_df["Fare"])
c = pd.cut(train_df["Fare"], bins=10)
def beautify(s):
    return str(s)[1:-1].replace(",", " ~ ")
b = c.apply(beautify)
plt.figure(figsize=(20, 10))
sns.countplot(x=b, hue=train_df["Survived"])
plt.xticks(rotation=30)
plt.legend(loc=1)
plt.title("Fare Plot圖")

In [ ]:
c = pd.cut(train_df["Age"], bins=10)
def beautify(s):
    return str(s)[1:-1].replace(",", " ~ ")
b = c.apply(beautify)
plt.figure(figsize=(20, 10))
sns.countplot(x=b, hue=train_df["Survived"])
plt.xticks(rotation=30)
plt.legend(loc=1)
plt.title("Fare Plot")

In [ ]:
# 收縮後的資料拆分成x_train, x_test
x_scale = datas_scale.iloc[:len(train_df)]
predict_scale = datas_scale.iloc[len(train_df):]

用另一種演算法比較
KNN (K-nearest Neighbors) 找最近鄰

[ https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html ]

log_loss越接近0越好,越小越好 \
相反地, neg_log_loss 為 -log_loss, 故越接近0越好,越大越好

KNN演算法
[ https://medium.com/@NorthBei/machine-learning-knn%E5%88%86%E9%A1%9E%E6%BC%94%E7%AE%97%E6%B3%95-b3e9b5aea8df ]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
clf = KNeighborsClassifier()
params = {
    "n_neighbors":range(5, 200)
}
grid = GridSearchCV(clf, params, cv=10, n_jobs=-1, scoring='neg_log_loss')
grid.fit(x_scale, y)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=11)
# 老師嘗試最好的成績是：11
clf.fit(x_scale, y)
pre = clf.predict(predict_scale)
ans = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
print(ans)
ans.to_csv("knn.csv", encoding="utf-8", index=False)

RamdomForest vs. KNeighbors

資料的準備、資料的整理, 是最重要的, 是我的天花板, \
演算法只是幫助逼近我想要的結果 \
不同\相同演算法 不應該差太多(1~2% ok)

1. 吃的資料量 \
    RF: 全部 \
    KNN: 附近k個 (局部)=> 適用資料量較少時 ex:推薦系統 \

2. 調整參數
    RF: 難度較高, 要靠賽,要不斷try, 找出最穩定的參數
    KNN: 較穩定, 沒有懸念

3. 對於特徵的重視程度- 特徵係數(重要性)：
    RF: 有。欄位重要程度有差別
    KNN: 無 。套公式算距離,每個距離都一視同仁, 沒有對每個特徵做分層

4. (線性)相關係數：

    ex:feature_importances_

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html ]


